**보충할 부분들:**
    - queryset (post) 더 많이 생성 (lecture11 전 lec에서 나온내용임분
    - tag_set을 admin에 등록 아직 안했음.
    - tag_set 데이터들 생성 아직 안했음.                                   
    - startapp shop 안만들어뒀음.
    - 마지막 10분 tag_set가지고 ManyToManyField manipulate하는 함수들 실습 안했음.

## ForeignKey

In [1]:
Comment

blog.models.Comment

In [2]:
comment = Comment.objects.first()

In [3]:
from django.db import connection
connection.queries[-1]

{'sql': 'SELECT "blog_comment"."id", "blog_comment"."post_id", "blog_comment"."author", "blog_comment"."message", "blog_comment"."created_at", "blog_comment"."updated_at" FROM "blog_comment" ORDER BY "blog_comment"."id" ASC  LIMIT 1',
 'time': '0.000'}

In [4]:
comment.post_id

1

In [5]:
print(comment.post)
print(Post.objects.get(id=comment.post_id))

ANALYSIS: Martin Luther King Jr. Day spotlights diversity in the 2020 Democratic field
ANALYSIS: Martin Luther King Jr. Day spotlights diversity in the 2020 Democratic field


## OneToOneField  (39:50)

In [6]:
Profile

accounts.models.Profile

In [7]:
profile = Profile.objects.first()

In [8]:
from django.db import connection
connection.queries[-1]

{'sql': 'SELECT "accounts_profile"."id", "accounts_profile"."user_id", "accounts_profile"."phone_number", "accounts_profile"."address" FROM "accounts_profile" ORDER BY "accounts_profile"."id" ASC  LIMIT 1',
 'time': '0.000'}

In [9]:
profile.user_id

1

In [10]:
profile.user

<User: celine>

In [12]:
connection.queries[-1]

{'sql': 'SELECT "auth_user"."id", "auth_user"."password", "auth_user"."last_login", "auth_user"."is_superuser", "auth_user"."username", "auth_user"."first_name", "auth_user"."last_name", "auth_user"."email", "auth_user"."is_staff", "auth_user"."is_active", "auth_user"."date_joined" FROM "auth_user" WHERE "auth_user"."id" = 1',
 'time': '0.001'}

Lazy:
    본 model instance를 획득할 때야
    ForeignKey, OneToOneField의 관계에 있는 레코드를 같이 fetch해 온다. (최대한 게으르게 획득해온다는 것이 포인트.)

#### 참고: auth.User model과 관계를 맺을 때

장고는 사용자 인증에 사용되는 User model 변경을 지원한다. 
비추천 방법1: user = models.OneToOneField(User)
비추천 방법2: user = models.OneToOneField('auth.User')
    
추천 방법3: 
user = models.OneToOneField(settings.AUTH_USER_MODEL)

참고 doc: 
https://docs.djangoproject.com/en/2.1/topics/auth/customizing/#auth-custom-user

# ForeignKey 에 관한 내용들:

##  1. ForeignKey.on_delete 옵션 45:57
#### Post가 지워지면 해당 post의 comments는 어떻게 할 것인가?

django 2.0 에서는 default = CASCADE 아님.

## 2. ForeignKey에서 related_name 지정의 필요성 
#### 1:N 관계에서 1측에서 N측으로 접근 시의 속성명 : "모델명소문자_set"
###### (1측 = post, N측 = comment)

In [17]:
# 실습을 위해 comment 우선 생성
Comment.objects.create(post = post, author='auth1', message='comment1')
Comment.objects.create(post = post, author='auth2', message='comment2')
Comment.objects.create(post = post, author='auth3', message='comment3')


<Comment: Comment object (6)>

In [21]:
# 첫 번째 post의 댓글을 가져오고 싶다면:
post = Post.objects.first()

## 방법1. N측에서 직접 접근 
print(Comment.objects.filter(post=post))  

print('\n--------same result--------\n')

## 방법2. 1측에서 related_name을 이용해서 N측 접근
print(post.comment_set.all())

<QuerySet [<Comment: Comment object (4)>, <Comment: Comment object (5)>, <Comment: Comment object (6)>]>

--------same result--------

<QuerySet [<Comment: Comment object (4)>, <Comment: Comment object (5)>, <Comment: Comment object (6)>]>


### 2-1. related_name 중복되는 경우: 한쪽 model에서 related_name(modelname_set)을 포기하거나 중복을 없애주거나

'post' model에도 'user' 항목이 있고, 'shop' model에도 'user'항목이 있을 때, related_name을 이용해서 N측 접근을 하려고하면, 그 이름이 겹치므로 error남. 

따라서 한쪽 model에서 'user'을 변경하거나, 또는 'user'의 필드값에 related_name = 'new_name'을 지정해줘서 clash를 피할 수 있음. 대신 new_name을 지정한 model은 'user_set'을 이용해서 데이터 접근을 할 수 없음. 

In [22]:
# blog/models.py
class Post(models.Model):
    user = models.ForeignKey(settings.AUTH_USER_MODEL, related_name = 'blog_post_set')

# shop/models.py
class Post(models.Model):
    user = models.ForeignKey(settings.AUTH_USER_MODEL, related_name='shop_post_set')
    
## 이런식으로 related_name = 'new_name'을 붙여줌으로써 중복되는 related_name의 문제를 해결할 수 있다.

NameError: name 'models' is not defined